In [1]:
from bond_valence_processor import BondValenceProcessor

/Users/chunhuili/opt/anaconda3/envs/mp/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/chunhuili/opt/anaconda3/envs/mp/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [20]:
cations = ['B'] # a list of cation species
my_api_key = "YgzOEXsODWlsR0J9P5aSjX2CxHuZX9Zv"
algos = ['shgo', 'brute', 'diff', 'dual_annealing', 'direct']
processor = BondValenceProcessor(my_api_key, algos)
    
for cation in cations:
    processor.process_cation_system(cation)

# Correct SSH format for GitHub is:
# git@github.com:username/repo.git
# So for your case it should be:
# git@github.com:nodameCL/BondValenceParametersFit.git

# First check if you have SSH access set up properly:
# 1. Generate SSH key if you haven't:
# ssh-keygen -t ed25519 -C "your_email@example.com"
# (press enter to accept default location)

# 2. Add SSH key to ssh-agent:
# eval "$(ssh-agent -s)"
# ssh-add ~/.ssh/id_ed25519

# 3. Copy public key to clipboard:
# pbcopy < ~/.ssh/id_ed25519.pub

# 4. Add SSH key to GitHub account:
# - Go to GitHub -> Settings -> SSH and GPG keys
# - Click "New SSH key"
# - Paste your public key

# Then try cloning with:
# git clone git@github.com:nodameCL/BondValenceParametersFit.git

# If you still get errors, check:
# - Your SSH config (~/.ssh/config)
# - Network connectivity
# - GitHub status (status.github.com)

# After cloning and making changes, follow these steps to push code:

# 1. Add changed files to staging
# git add .

# 2. Commit changes with a message
# git commit -m "Your commit message describing changes"

# 3. Push changes to remote repository
# git push origin main  # or 'master' depending on your default branch

# If you get errors about being behind remote, first pull latest changes:
# git pull origin main

# Then resolve any merge conflicts and push again:
# git push origin main

# To create a new branch and push:
# git checkout -b new-branch-name
# git push -u origin new-branch-name


# download data from materials project

## X-O containing materials

In [7]:
import json
import os 
import numpy as np
# get matID to possible species dict 
def get_possible_species_perID(save_dir, docs):
    import json 
    dict_matID_possible_species = {} 
    for doc in docs: 
        if doc.possible_species != []: 
            dict_matID_possible_species[doc.material_id] = doc.possible_species

    if not os.path.exists(f'{save_dir}/params'): 
        os.makedirs(f'{save_dir}/params')
        
    with open(f'{save_dir}/params/dict_matID_possible_species.json', 'w') as fopen: 
        json.dump(dict_matID_possible_species, fopen)

    return list(dict_matID_possible_species.keys())

In [2]:
my_api_key = "YgzOEXsODWlsR0J9P5aSjX2CxHuZX9Zv"
algos = ['shgo', 'brute', 'diff', 'dual_annealing', 'direct']

In [7]:
import numpy as np 
check_list = np.loadtxt('/Users/chunhuili/SynologyDrive/00Research/00_MINES/00_mp_data_process/no_solu/direct.txt', dtype=str).tolist()

In [ ]:
# bond_valence_processor.py
from tqdm import tqdm
import os
import numpy as np
import json
from mp_api.client import MPRester
from BVparams_search import TheoreticalBondValenceSolver
from BVparams_search import BVParamSolver

class BondValenceProcessor:
    def __init__(self, api_key, algos):
        self.api_key = api_key
        self.algos = algos
        
    def process_cation_system(self, cation):
        """Process a single cation-O system"""
        print(f'start calculating {cation}-O system:')
        
        # Download dataset
        docs = self.download_materials_data(cation)
        
        # Setup directories and get material IDs
        res_dir = f'res/{cation}O'
        mids = self.get_possible_species_perID(res_dir, docs)
        
        # Get bonding data
        bonds_docs = self.download_bonding_data(mids)
        
        # Initialize data structures
        dict_sij_perMatID = {}
        dict_charges_perMatID = {}
        
        # Handle previously solved cases
        solved_materID, no_solu = self.get_previous_results(res_dir)
        
        # Initialize solver
        sij_solver = TheoreticalBondValenceSolver(
            species_matID_path=f'{res_dir}/params/dict_matID_possible_species.json'
        )
        
        # Process each material
        self.process_materials(bonds_docs, solved_materID, sij_solver, dict_sij_perMatID, 
                         dict_charges_perMatID, no_solu, res_dir, cation)
        
        # Save results
        self.save_results(res_dir, dict_sij_perMatID, dict_charges_perMatID)

    def download_materials_data(self, cation):
        """Download materials data from Materials Project"""
        with MPRester(api_key=self.api_key) as mpr:
            return mpr.materials.summary.search(
                elements=[cation, 'O'],
                energy_above_hull=(0.000, 0.05),
                fields=['material_id', 'possible_species']
            )

    def download_bonding_data(self, mids):
        """Download bonding data from Materials Project"""
        with MPRester(api_key=self.api_key) as mpr:
            return mpr.materials.bonds.search(
                material_ids=mids,
                fields=['material_id', 'structure_graph', 'formula_pretty']
            )

    def get_previous_results(self, res_dir):
        """Get previously solved cases"""
        if os.path.exists(f'{res_dir}/R0Bs/shgo'):
            solved_sol = os.listdir(f'{res_dir}/R0Bs/shgo/')
            solved_materID = [e.split('.txt')[0] for e in solved_sol]
            if '.ipynb_checkpoints' in solved_materID:
                solved_materID.remove('.ipynb_checkpoints')
            
            no_solu = np.loadtxt(f'{res_dir}/no_solu/shgo.txt', dtype=str).tolist()
            nosol_IDs = [e[0] for e in no_solu]
            nosol_IDs = list(set(nosol_IDs))
            solved_materID += nosol_IDs
        else:
            solved_materID = []
            no_solu = []
        
        return solved_materID, no_solu

    def process_materials(self, bonds_docs, solved_materID, sij_solver, dict_sij_perMatID,
                         dict_charges_perMatID, no_solu, res_dir, cation):
        """Process each material in the dataset"""
        for Li_mater in tqdm(bonds_docs):
            matID = Li_mater.material_id
            reduced_formula = Li_mater.formula_pretty
            cur_struct = Li_mater.structure_graph.structure
            
            # Get Sij values
            cur_network_valence_dict, cur_bond_type_list, cur_bondL_dict, dict_charge = sij_solver.get_sij(
                matID, cur_struct, Li_mater.structure_graph
            )
            
            dict_sij_perMatID[matID] = cur_network_valence_dict
            dict_charges_perMatID[matID] = dict_charge

            if matID in solved_materID:
                continue
                
            self.process_algorithm_results(cur_network_valence_dict, no_solu, res_dir,
                                    cation, matID, reduced_formula, cur_bond_type_list,
                                    cur_bondL_dict)

    def process_algorithm_results(self, cur_network_valence_dict, no_solu, res_dir,
                                cation, matID, reduced_formula, cur_bond_type_list,
                                cur_bondL_dict):
        """Process results using different algorithms"""
        for alg in self.algos:
            if not cur_network_valence_dict:
                no_solu.append((matID, cation, 'O', reduced_formula, 'no_network_sol'))
                np.savetxt(f'{res_dir}/no_solu/{alg}.txt', no_solu, fmt='%s')
                continue
                
            bv_solver = BVParamSolver(save_dir=res_dir, algo=alg, no_sol=no_solu)
            
            new_R0_B_LiO = bv_solver.solve_R0Bs(
                cation=cation,
                anion='O',
                bond_type_list=cur_bond_type_list,
                networkValence_dict=cur_network_valence_dict,
                bondLen_dict=cur_bondL_dict,
                materID=matID,
                chem_formula=reduced_formula,
                R0_bounds=(0, 5),
            )
            
            if new_R0_B_LiO:
                np.savetxt(f'{res_dir}/R0Bs/{alg}/{matID}.txt', new_R0_B_LiO)

    def save_results(self, res_dir, dict_sij_perMatID, dict_charges_perMatID):
        """Save final results"""
        with open(f'{res_dir}/dict_sijs.json', 'w') as fopen:
            json.dump(dict_sij_perMatID, fopen)
        
        with open(f'{res_dir}/dict_charges.json', 'w') as fopen:
            json.dump(dict_charges_perMatID, fopen)

# Main processing
if __name__ == "__main__":
    my_api_key = "YgzOEXsODWlsR0J9P5aSjX2CxHuZX9Zv"
    algos = ['shgo', 'brute', 'diff', 'dual_annealing', 'direct']
    processor = BondValenceProcessor(my_api_key, algos)
    
    for cation in cations:
        processor.process_cation_system(cation)

start calculating Ti-O system:


Retrieving SummaryDoc documents:   0%|          | 0/2284 [00:00<?, ?it/s]

Retrieving BondingDoc documents:   0%|          | 0/2235 [00:00<?, ?it/s]

 31%|███       | 685/2235 [1:01:49<4:29:04, 10.42s/it] /global/home/users/chli/.local/lib/python3.9/site-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
 55%|█████▌    | 1237/2235 [3:46:27<10:49,  1.54it/s]    spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 1 tolerance = 9.500000e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 2 tolerance = 9.025000e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 3 tolerance = 8.573750e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 4 tolerance = 8.145062e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 5 tolerance = 7.737809e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 6 tolerance = 7.350919e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 7 tolerance =